## Libreria necesarias

In [1]:
import pandas as pd
import numpy as np
import keras_metrics
import tensorflow as tf
import shutil, os, sys
import matplotlib.pyplot as plt



from tensorflow import keras
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D
from tensorflow.keras import optimizers, layers
from tensorflow.keras import utils as k
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing import image_dataset_from_directory 
from tensorflow.keras.layers import BatchNormalization

from tensorflow.keras.applications.inception_v3 import InceptionV3,preprocess_input




In [2]:
rutaEntreno='fruits-360\Training'


dimensiones=[100,100]

imagenes_train= image_dataset_from_directory(rutaEntreno, labels='inferred', label_mode='categorical', color_mode='rgb', batch_size=1000, image_size=(dimensiones[0],dimensiones[1]), validation_split=0.3,subset="training",seed=1234)

imagenes_val= image_dataset_from_directory(rutaEntreno, labels='inferred', label_mode='categorical', color_mode='rgb', batch_size=1000, image_size=(dimensiones[0],dimensiones[1]), validation_split=0.3,subset="validation",seed=1234)


clases= imagenes_train.class_names

Found 67692 files belonging to 131 classes.
Using 47385 files for training.
Found 67692 files belonging to 131 classes.
Using 20307 files for validation.


In [31]:
#x_t=[]
#y_t=[]
#for batch, labels in imagenes_train:
#    x_t.append(batch.numpy())
#    y_t.append(labels.numpy())
#x_train=np.array(x_t)
#y_train=np.array(y_t)    
#
#print('cambio')
#
#x_v=[]
#y_v=[]
#for batch, labels in imagenes_val:
#    x_v.append(batch.numpy())
#    y_v.append(labels.numpy())
#
#x_val=np.array(x_v)
#y_val=np.array(y_v)

In [3]:
#aplicamos el preprocesamiento necesario a las entradas
image_batch, label_batch = next(iter(imagenes_train))
image_batchV, label_batchV = next(iter(imagenes_val))

In [11]:
inception= InceptionV3(weights='imagenet', include_top=False)

In [12]:
aumento = keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal", input_shape=(dimensiones[0],dimensiones[1],3)),
    layers.experimental.preprocessing.RandomFlip("vertical", input_shape=(dimensiones[0],dimensiones[1],3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    #layers.experimental.preprocessing.RandomZoom(0.1),
    #layers.experimental.preprocessing.Rescaling(1./255, input_shape=(dimensiones[0],dimensiones[1],3))
  ]
)

inception.trainable=False
inputs=keras.Input(shape=(dimensiones[0], dimensiones[1],3))
x=aumento(inputs)
x=inception(x, training=False)
x=GlobalAveragePooling2D()(x)
outputs=Dense(len(clases))(x)
model=Model(inputs,outputs)


In [13]:
x_entreno=preprocess_input(image_batch)
y_entreno=preprocess_input(label_batch)
x_validacion=preprocess_input(image_batchV)
y_validacion=preprocess_input(label_batchV)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.fit(x=x_entreno, y=y_entreno,validation_data=(x_validacion,y_validacion), epochs= 20,shuffle=True)

Epoch 1/20
32/32 [==============================] - 60s 1s/step - loss: -1333.0061 - categorical_accuracy: 0.0082 - val_loss: -1597.0153 - val_categorical_accuracy: 0.0050
Epoch 2/20
32/32 [==============================] - 27s 855ms/step - loss: -1610.2610 - categorical_accuracy: 0.0093 - val_loss: -1662.2687 - val_categorical_accuracy: 0.0050
Epoch 3/20
32/32 [==============================] - 26s 813ms/step - loss: -1678.0860 - categorical_accuracy: 0.0137 - val_loss: -1715.8528 - val_categorical_accuracy: 0.0080
Epoch 4/20
32/32 [==============================] - 26s 816ms/step - loss: -1740.3039 - categorical_accuracy: 0.0073 - val_loss: -1826.9626 - val_categorical_accuracy: 0.0050
Epoch 5/20
32/32 [==============================] - 23s 723ms/step - loss: -1829.1284 - categorical_accuracy: 0.0158 - val_loss: -1840.0162 - val_categorical_accuracy: 0.0100
Epoch 6/20
32/32 [==============================] - 23s 715ms/step - loss: -1841.1641 - categorical_accuracy: 0.0068 - val_loss:

In [15]:
inception.trainable=True
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x=x_entreno, y=y_entreno,validation_data=(x_validacion,y_validacion), epochs= 10,shuffle=True)

Epoch 1/10
 8/32 [======>.......................] - ETA: 1:01 - loss: -1749.2449 - accuracy: 4.8828e-04

KeyboardInterrupt: 

In [34]:
# add a global spatial average pooling layer
x = inception.output
x = GlobalAveragePooling2D()(x)
x=layers.Flatten()(x)
# let's add a fully-connected layer
x = Dense(500, activation='relu')(x)
x= layers.Dropout(0.5)(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(len(clases), activation='softmax')(x)

model= Model(inputs=inception.input,outputs=predictions)



In [35]:


#x_entreno=preprocess_input(image_batch)
#y_entreno=preprocess_input(label_batch)
#
#x_validacion=preprocess_input(image_batchV)
#y_validacion=preprocess_input(label_batchV)
#
#
##Congelo las capas de convolucion de InceptionV3
#for layer in inception.layers:
#    layer.trainable = False
#
#model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
#model.fit(x=x_entreno, y=y_entreno,validation_data=(x_validacion,y_validacion), epochs= 50,shuffle=True, use_multiprocessing=True)

In [36]:
#for i, layer in enumerate(inception.layers):
#   print(i, layer.name)

In [37]:
#for layer in model.layers[:200]:
#   layer.trainable = False
#for layer in model.layers[200:]:
#   layer.trainable = True
#
#from tensorflow.keras.optimizers import SGD
#model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy',metrics=['accuracy'])
#model.fit(x=x_entreno, y=y_entreno,validation_data=(x_validacion,y_validacion), epochs= 100,shuffle=True, use_multiprocessing=True, steps_per_epoch=100)

In [43]:
aumento = keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal", input_shape=(dimensiones[0],dimensiones[1],3)),
    layers.experimental.preprocessing.RandomFlip("vertical", input_shape=(dimensiones[0],dimensiones[1],3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    #layers.experimental.preprocessing.RandomZoom(0.1),
    layers.experimental.preprocessing.Rescaling(1./255, input_shape=(dimensiones[0],dimensiones[1],3))
  ]
)

#Normalizar= layers.experimental.preprocessing.Rescaling(1./255, input_shape=(dimensiones[0],dimensiones[1],1))
pool= 3#CON UN POOL MÁS GRANDE SE REDUCEN LOS PESOS
pd= layers.ZeroPadding2D(padding=2) #PADDING
model2= Sequential([
    aumento,
    #BatchNormalization(),
    #Normalizar,
    pd,
    layers.Conv2D(filters=16,kernel_size=5,padding='same',activation='relu', strides=1),
    #layers.Conv2D(filters=16,kernel_size=5,padding='same',activation='relu', strides=1),
    #layers.Conv2D(filters=16,kernel_size=5,padding='same',activation='relu', strides=1),
    layers.MaxPooling2D(pool_size=pool),
    BatchNormalization(),
    layers.Conv2D(filters=32,kernel_size=3,padding='same',activation='relu', strides=1),
    #layers.Conv2D(filters=32,kernel_size=3,padding='same',activation='relu', strides=1),
    #layers.Conv2D(filters=32,kernel_size=3,padding='same',activation='relu', strides=1),
    layers.MaxPooling2D(pool_size=pool),
    BatchNormalization(),
    #layers.Conv2D(filters=64,kernel_size=3,padding='same',activation='relu', strides=1),
    #layers.Conv2D(filters=64,kernel_size=3,padding='same',activation='relu', strides=1),
    layers.Conv2D(filters=64,kernel_size=3,padding='same',activation='relu', strides=1),
    BatchNormalization(),
    layers.Conv2D(filters=128,kernel_size=3,padding='same',activation='relu', strides=1),
    layers.MaxPooling2D(pool_size=pool),
    BatchNormalization(),

    #layers.Conv2D(filters=128,kernel_size=3,padding='same',activation='relu', strides=1),
    #layers.MaxPooling2D(pool_size=pool),

    layers.Flatten(),
    Dense(500,'relu'),
    layers.Dropout(0.5),
    Dense(len(clases),'softmax')

])

model2.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_17 (Sequential)   (None, 100, 100, 3)       0         
_________________________________________________________________
zero_padding2d_9 (ZeroPaddin (None, 104, 104, 3)       0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 104, 104, 16)      1216      
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 34, 34, 16)        0         
_________________________________________________________________
batch_normalization_31 (Batc (None, 34, 34, 16)        64        
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 34, 34, 32)        4640      
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 11, 11, 32)      

In [ ]:
opti= optimizers.RMSprop(learning_rate=0.0025, momentum=0.5)#learning_rate=0.05,epsilon=0.8
vueltas=5
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)

model2.compile(optimizer=opti, loss='categorical_crossentropy', metrics=['accuracy'])
historico=model2.fit(x=image_batch, y=label_batch,validation_data=(image_batchV,label_batchV), epochs= 1000,shuffle=True)

In [17]:
rutaTest='fruits-360/Test'
imagenes_test= image_dataset_from_directory(rutaTest, labels='inferred', label_mode='categorical', color_mode='rgb',shuffle=False,batch_size=32, image_size=(dimensiones[0],dimensiones[1]))

h=imagenes_test.as_numpy_iterator()
salidas=[]
lista=[]
for image in h:
    for i in range(len(image[0])):
        lista.append(image[0][i])
        salidas.append(image[1][i])

Found 22688 files belonging to 131 classes.


In [23]:
tensores=[]
for i in range(len(lista)):
    tensores.append(tf.convert_to_tensor(lista[i]))


In [30]:
tensores[0].shape

TensorShape([100, 100, 3])

In [27]:
for i in range(len(tensores)):
    print(i)
    pr=model.predict(tensores[i])
    a=np.argmax(pr)
    print("imagen: ", i, 'pertenece a: ', clases[a])

0


ValueError: in user code:

    C:\Users\Festr\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1478 predict_function  *
        return step_function(self, iterator)
    C:\Users\Festr\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1468 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\Festr\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\Festr\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\Festr\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\Festr\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1461 run_step 

In [39]:
numeros=[4,2,6,7,4,3,9]
a=sorted(numeros, reverse=True)
#print(numeros.index(a[0]))
for i in range(3):
    print(i)

0
1
2
